In [ ]:
###### Case0 #####
import deepxde as dde
import numpy as np

R = 10.0
C = 1.0
U = 1.0

def ode_system(x, I):
    """ODE for dI/dt + I/(RC) = 0"""
    dI_dx = dde.grad.jacobian(I, x)
    return dI_dx + I / (R * C)

def func(x):
    """Exact solution for dI/dt + I/(RC) = 0"""
    # The exact solution
    return (U / R) * np.exp(-x / (R * C))

geom = dde.geometry.TimeDomain(0, 10)

# Define boundary conditions using DirichletBC
def boundary(_, on_initial):
    return on_initial

# Define the initial condition function
def initial_condition(x):
    return U / R  # Initial condition is constant

ic = dde.icbc.IC(geom, lambda x: U/R, boundary,component=0)  # Initial condition

data = dde.data.PDE(geom, ode_system, [ic], 35, 1, solution=func, num_test=10)

layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)
